# Box–Jenkins method

[Wiki Pages](https://en.wikipedia.org/wiki/Box–Jenkins_method)

[How to Create an ARIMA Model for Time Series Forecasting with Python](https://machinelearningmastery.com/arima-for-time-series-forecasting-with-python/)

[AR(I)MA时间序列建模过程——步骤和python代码](https://www.jianshu.com/p/cced6617b423)

[Python 3中使用ARIMA进行时间序列预测的指南](https://www.howtoing.com/a-guide-to-time-series-forecasting-with-arima-in-python-3)

[A comprehensive beginner’s guide to create a Time Series Forecast (with Codes in Python)](https://www.analyticsvidhya.com/blog/2016/02/time-series-forecasting-codes-python/)

In [1]:
from __future__ import print_function, division, with_statement

In [2]:
!ls /home/kesci/input/

In [3]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [4]:
plt.style.use('fivethirtyeight')
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [5]:
import statsmodels.api as sm

In [6]:
data = pd.read_csv("/home/kesci/input/foshan9801/foshan",sep='\t')

In [7]:
#data['Hosp_kind'] = data['Hosp_kind'].apply(lambda x: 4 if x==9 else x)
data['Date'] = data['Date'].apply(pd.to_datetime,format='%Y%m%d')

In [8]:
data.tail(10)

In [9]:
fig, ax = plt.subplots(nrows=3, ncols=2,figsize=plt.figaspect(0.2))
fig.tight_layout()
for i,row in enumerate(ax):
    for j,axs in enumerate(row):
        data[(data['In_kind']==j+1) & (data["Hosp_kind"]==i+1) ].groupby(['Date'])['Cost'].sum().plot(ax=axs,figsize=(15,6))
        axs.set(title='In_kind={},Hosp={}'.format(j+1,i+1))
        #axs.axes.get_xaxis().set_visible(False)

plt.show()

In [10]:
test = data[(data['In_kind']==1)].groupby(['Date'])['Cost'].sum()
test

In [11]:
test.drop(test.index[-1],inplace=True)
print(test.index[1])
test.values

In [12]:
fig, ax = plt.subplots(figsize=(20,6))
ax.plot(test)

In [13]:
import warnings
warnings.filterwarnings("ignore")
import statsmodels.tsa.stattools as st
order = st.arma_order_select_ic(test.values,max_ar=5,max_ma=5,ic=['aic', 'bic', 'hqic'])
order.bic_min_order


In [14]:
from statsmodels.tsa.stattools import adfuller
def test_stationarity(timeseries):
    dftest = adfuller(timeseries, autolag='AIC')
    print(dftest)
    return dftest[1]

In [15]:
def produce_diffed_timeseries(df, diffn):
    if diffn != 0:
        df['diff'] = df[df.columns[1]].apply(lambda x:float(x)).diff(diffn)
    else:
        df['diff'] = df[df.columns[1]].apply(lambda x:float(x))
    df.dropna(inplace=True) #差分之后的nan去掉
    return df


In [16]:

if test_stationarity(test.values) < 0.01:
    print('平稳，不需要差分')
else:
    diffn = best_diff(train, maxdiff = 8)
    train = produce_diffed_timeseries(train, diffn)
    print('差分阶数为'+str(diffn)+'，已完成差分')



In [17]:
from statsmodels.tsa.arima_model import ARIMA

In [18]:
model = ARIMA(test.values, order=(4, 0, 5))  

In [19]:
results_AR = model.fit(disp=-1)  

fig, ax = plt.subplots(figsize=(15,6))
ax.plot(test.values)
ax.plot(results_AR.fittedvalues)
#ax.plot(np.concatenate((results_AR.fittedvalues,results_AR.predict(150,177))), color='red')
#plt.plot(results_AR.predict(160,177), color='green')
#plt.title('RSS: %.4f'% sum((results_AR.fittedvalues-test.values)**2))

In [20]:
results_AR.predict(160,177)

In [1]:
mod = sm.tsa.statespace.SARIMAX(test.values[0:140],
                                order=(2, 0, 1),
                                seasonal_order=(1, 1, 1, 12),
                                enforce_stationarity=False,
                                enforce_invertibility=False)
res=mod.fit()
res.aic

In [3]:
import itertools
p = d = q = range(0, 7)

# Generate all different combinations of p, q and q triplets
pdq = list(itertools.product(p, d, q))

# Generate all different combinations of seasonal p, q and q triplets
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]
dict_t={}

for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(test.values[0:200],
                                            order=param,
                                            seasonal_order=param_seasonal,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)

            results = mod.fit()
            dict_t[param_seasonal]=results.aic
            print('ARIMA{}x{}12 - AIC:{}'.format(param, param_seasonal, results.aic))
        except:
            continue
print(dict_t.sort())

In [32]:
result = mod.fit()

In [33]:
pred = result.predict(140,177)

In [34]:
pred = pd.Series(pred,index=range(139,177))

In [35]:
fig, ax = plt.subplots(figsize=(15,6))
ax.plot(test.values)
ax.plot(result.fittedvalues)
ax.plot(pred)
#pred.predicted_mean.plot(ax=ax)
#ax.plot(np.concatenate((result.fittedvalues,result.predict(140,177))), color='red')

In [36]:
print(result.summary().tables[1])

In [37]:
result.plot_diagnostics(figsize=(15, 12))
plt.show()

----

# Work on days 

In [38]:
data = pd.read_csv("./ZhanJiang_all",sep='\t')

In [ ]:
data['Hosp_kind'] = data['Hosp_kind'].apply(lambda x: 4 if x==9 else x)

In [ ]:
data['Date'] = data['Date'].apply(pd.to_datetime,format='%Y%m%d')

In [ ]:
test = data[(data['In_kind']==1) & (data["Hosp_kind"]==3) ].groupby(['Date'])['Cost'].sum()

### test month pattern

In [ ]:
fig, ax = plt.subplots(figsize=(15,6))
ax.plot(test[0:20])
#ax.plot(test[61:121])

---

## ARIMA

In [ ]:
model = ARIMA(test.values, order=(5, 0, 5))  

In [ ]:
results_AR = model.fit(disp=-1)  

fig, ax = plt.subplots(figsize=(15,6))
ax.plot(test.values)
ax.plot(results_AR.fittedvalues)

In [ ]:
fig, ax = plt.subplots(figsize=(15,6))
ax.plot(test.values[1000:])
ax.plot(results_AR.fittedvalues[1000:])

### test for ARIMA

In [ ]:
test.shape

In [ ]:
model = ARIMA(test.values[0:1100], order=(3, 0, 4))  
results_AR = model.fit(disp=-1)  

In [ ]:
pred = results_AR.predict(1100,test.shape[0])

In [ ]:
pred3 = pd.Series(pred,index=range(1099,test.shape[0]))
pred2 = pd.Series(pred,index = range(99,test.shape[0]-1100+99+1))

In [ ]:
fig, ax = plt.subplots(figsize=(15,6))
ax.plot(test.values)
ax.plot(results_AR.fittedvalues)
ax.plot(pred3)

In [ ]:
fig, ax = plt.subplots(figsize=(15,6))
ax.plot(test.values[1000:])
ax.plot(results_AR.fittedvalues[1000:])
ax.plot(pred2)

## SARIMAX

In [ ]:
mod = sm.tsa.statespace.SARIMAX(test.values[0:1100],
                                order=(3, 0, 4),
                                seasonal_order=(1, 1, 1, 14),
                                enforce_stationarity=False,
                                enforce_invertibility=False)

In [ ]:
result = mod.fit()

In [ ]:
pred = result.predict(1100,test.shape[0])

In [ ]:
pred3 = pd.Series(pred,index=range(1099,test.shape[0]))

In [ ]:
pred3.plot()

In [ ]:
pred2 = pd.Series(pred,index = range(99,test.shape[0]-1100+99+1))

In [ ]:
fig, ax = plt.subplots(figsize=(15,6))
ax.plot(test.values[1000:])
ax.plot(result.fittedvalues[1000:])
ax.plot(pred2)

In [ ]:
fig, ax = plt.subplots(figsize=(15,6))
ax.plot(test.values)
ax.plot(result.fittedvalues)
ax.plot(pred3)

In [ ]:
import itertools

In [ ]:
d = range(0,2)
p = q = range(0, 10)

# Generate all different combinations of p, q and q triplets
pdq = list(itertools.product(p, d, q))

# Generate all different combinations of seasonal p, q and q triplets
seasonal_pdq = [(x[0], x[1], x[2], 14) for x in list(itertools.product(p, d, q))]

In [ ]:
#warnings.filterwarnings("ignore") # specify to ignore warning messages
min_value = 50000

for param in pdq:
    for param_seasonal in seasonal_pdq:
        mod = sm.tsa.statespace.SARIMAX(test.values,
                                        order=param,
                                        seasonal_order=param_seasonal,
                                        enforce_stationarity=False,
                                        enforce_invertibility=False)

        results = mod.fit()
        if min_value > results.aic :
            min_value = results.aic
            pm,pm_s = param, param_seasonal
        
        print('ARIMA{}x{}12 - AIC:{}'.format(param, param_seasonal, results.aic))
#         except:
#             continue

In [ ]:
pm,pm_s